In [1]:
using LowRankModels, Random, LinearAlgebra, Plots, Statistics, CSV

In [2]:
data = convert(Matrix, CSV.read("cleaned_data_notencoded.csv"))
n, d = size(data)
age = data[:,1]
data_int = convert(Matrix{Integer}, data[:,2:d])
data = hcat(age, data_int);

In [4]:
function partitionTrainTest(data, at = 0.8)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    data[train_idx,:], data[test_idx,:]
end

train, test = partitionTrainTest(data, 0.7);

In [5]:
x_train = train[:,1:d-1]
y_train = train[:,d]

x_test = test[:,1:d-1]
y_test = test[:,d];

In [6]:
n_train, d = size(train);

In [15]:
k = 3
X = Array{Float64}(undef, k, d)
Y = Array{Float64}(undef, k, d)
no_bootstrap = 1
for i in 1:no_bootstrap
    print("At bootstrap: ", i, "\n")
    Ω = findall(rand(n_train,d) .< .1)     
    loss = HuberLoss()
    #loss = Loss[QuadLoss(), BvSLoss(), LogLoss(), LogLoss(), BvSLoss(),
    #LogLoss(), LogLoss(), BvSLoss(), BvSLoss(), BvSLoss(), 
    #BvSLoss(), BvSLoss(), BvSLoss(), BvSLoss(), LogLoss()]
    reg = QuadReg(.001) # a tiny bit of quadratic regularization
    glrm = GLRM(train, loss, reg, reg, k, obs=Ω) 
    add_offset!(glrm)      # adds an offset to the model
    init_svd!(glrm)
    X, Y, ch = fit!(glrm)
    X, Y, ch = fit!(glrm)
    X, Y, ch = fit!(glrm)
    X, Y, ch = fit!(glrm)
    X, Y, ch = fit!(glrm)
    Y += 1/no_bootstrap * Y
    X += 1/no_bootstrap * X
end

At bootstrap: 1
Fitting GLRM
Iteration 10: objective value = 169.24020803057113
Iteration 20: objective value = 109.03761967921189
Iteration 30: objective value = 84.35400821176141
Iteration 40: objective value = 72.44446491589274
Iteration 50: objective value = 67.18458902787555
Iteration 60: objective value = 63.12997084678785
Iteration 70: objective value = 60.39203992426055
Iteration 80: objective value = 58.190492452749154
Iteration 90: objective value = 55.95125967645378
Iteration 100: objective value = 54.2753826601139
Fitting GLRM
Iteration 10: objective value = 51.66256128161153
Iteration 20: objective value = 49.88512779430381
Iteration 30: objective value = 47.759199388093684
Iteration 40: objective value = 45.76200619934789
Iteration 50: objective value = 44.585586089393985
Iteration 60: objective value = 43.507686499262284
Iteration 70: objective value = 42.36658685318229
Iteration 80: objective value = 41.13182300760114
Iteration 90: objective value = 39.80804937260375
It

In [16]:
Y

3×15 Array{Float64,2}:
 -0.204404  -0.35546  -0.483848  …  -0.683364   3.75956     0.158105
  0.258939   0.81458  -0.297476     23.6628     4.98428    -1.9693
  0.563714   1.66646   0.366309      1.82408   -0.0115509   1.25985

In [17]:
loss = HuberLoss()
#loss = [QuadLoss(), BvSLoss(), LogLoss(), LogLoss(), BvSLoss(),
#    LogLoss(), LogLoss(), BvSLoss(), BvSLoss(), BvSLoss(), 
#    BvSLoss(), BvSLoss(), BvSLoss(), BvSLoss(), LogLoss()]
reg = QuadReg(.001) # a tiny bit of quadratic regularization
glrm = GLRM(test, loss, reg, reg, k)  # the GLRM object stores the model, data, and parameter estimate
add_offset!(glrm)    # adds an offset to the model
init_svd!(glrm)
X_test, Y_test, ch = fit!(glrm);                          # alternating minimization

Fitting GLRM
Iteration 10: objective value = 1298.7668508742581
Iteration 20: objective value = 1294.2551020425785


In [18]:
y_pred = X'*Y[:,d]

695-element Array{Float64,1}:
 0.8190754390434782
 2.769834909336848
 2.2768567171357987
 1.215527718159327
 3.5681238033268308
 3.6324198084866905
 2.0311686418653907
 3.441395941445944
 0.0550772496655596
 8.910458915980556
 2.5197077933692755
 2.5197077933692755
 6.3764204332114245
 ⋮
 2.5243773573536967
 2.676097161395504
 2.5197077933692755
 4.276831928429
 2.5197077933692755
 2.5197077933692755
 3.441395941445944
 2.242006928084118
 1.8348192214301045
 1.3972137546930035
 2.5197077933692755
 2.5197077933692755

In [327]:
y_train

695-element Array{Real,1}:
 0
 1
 1
 1
 1
 0
 0
 1
 0
 1
 0
 0
 1
 ⋮
 0
 0
 0
 1
 1
 1
 1
 1
 0
 1
 1
 0